In [222]:
import googlemaps, os.path
import pandas as pd
import numpy as np

# https://matplotlib.org/basemap/users/examples.html
# https://meta.stackexchange.com/questions/2677/database-schema-documentation-for-the-public-data-dump-and-sede

In [223]:
sf_users = pd.read_csv(os.path.join("data","scifi.stackexchange.com", "Users.csv"))
sf_users.head()
sf_users.loc[:5, "Location"]

sf_tags = pd.read_csv(os.path.join("data","scifi.stackexchange.com", "Tags.csv"))
sf_posts = pd.read_csv(os.path.join("data","scifi.stackexchange.com", "Posts.csv"))

In [224]:
sf_tags.sort_values("Count", ascending = False).head()

,Unnamed: 0,Count,ExcerptPostId,Id,TagName,WikiPostId
47,47,12565,1811.0,130,story-identification,1810.0
168,168,5694,2679.0,533,harry-potter,2678.0
21,21,5305,1970.0,44,star-wars,1969.0
2,2,4319,1776.0,10,star-trek,1775.0
225,225,3352,4872.0,695,marvel,4871.0


In [225]:
# Chcemy zobaczyc ile pytan z tagiem "story-identification" uzyskalo odpowiedz.

sf_story_index = sf_posts.loc[:,"Tags"].str.contains("story-identification")\
.rename("story_identification").reset_index().query("story_identification == True").loc[:,"index"]
sf_story_index
number_of_questions = sf_posts.iloc[sf_story_index,:].query("PostTypeId == 1").shape[0]
number_of_answered_q = sf_posts.iloc[sf_story_index,:].query("PostTypeId == 1 and AcceptedAnswerId > 0").shape[0]

number_of_answered_q*100/number_of_questions

43.25507361719061

In [226]:
sf_users.head()

,Unnamed: 0,AboutMe,AccountId,CreationDate,DisplayName,DownVotes,Id,LastAccessDate,Location,ProfileImageUrl,Reputation,UpVotes,Views,WebsiteUrl
0,0,"<p>Hi, I'm not really a person.</p>\r\n\r\n<p>...",-1.0,2011-01-11T19:19:36.483,Community,15607,-1,2011-01-11T19:19:36.483,on the server farm,NaN,1,22476,0,http://meta.stackexchange.com/
1,1,<p>Developer on the Stack Overflow team. Find...,2.0,2011-01-11T19:50:40.620,Geoff Dalgas,0,2,2017-10-18T04:57:25.120,"Corvallis, OR",https://i.stack.imgur.com/nDllk.png,101,1,53,http://stackoverflow.com
2,2,<p>I am the Architecture Lead for Stack Overfl...,7598.0,2011-01-11T19:55:41.460,Nick Craver,0,3,2018-11-23T01:52:18.210,"Winston-Salem, NC",https://i.stack.imgur.com/nGCYr.jpg?s=128&g=1,101,4,29,https://nickcraver.com/blog/
3,3,"<p>co-founder of <a href=""https://airtable.com...",1998.0,2011-01-11T20:17:01.887,Emmett,0,4,2013-05-06T20:52:29.970,"San Francisco, CA",https://i.stack.imgur.com/d1oHX.jpg?s=128&g=1,101,0,26,http://minesweeperonline.com
4,4,"<p><a href=""http://blog.stackoverflow.com/2010...",29738.0,2011-01-11T20:18:52.493,Kevin Montrose,0,5,2018-07-11T16:01:10.277,"New York, NY, United States",NaN,101,43,28,http://kevinmontrose.com/


In [227]:
# przygotowanie do zrobienia mapy: dane

In [240]:
#!pip3 install lxml
import  lxml
adres = "https://en.wikipedia.org/wiki/List_of_countries_by_population_(United_Nations)"
res = pd.read_html(adres)

In [241]:
countries = res[1]
countries = countries.iloc[1:,[1,5]]
countries.columns = ["State_Country", "Population"]
countries.head()

,State_Country,Population
1,World,7550262101
2,China[a],1409517397
3,India,1339180127
4,United States,324459463
5,Indonesia,263991379


In [242]:
## States:

# adres = "https://en.wikipedia.org/wiki/List_of_states_and_territories_of_the_United_States_by_population"
# res = pd.read_html(adres)
# adres = "https://en.wikipedia.org/wiki/List_of_states_and_territories_of_the_United_States"
# res = pd.read_html(adres)
# states = res[0]
# states.columns = states.loc[0]
# states = states.iloc[2:,[0,1,5]]
# states.columns = ["State_Country","Abbr","Population"]
# states.head()

In [243]:
# # tabela pomocnicza do uzyskania w sf_users kolumny population
# states_abbr = states.iloc[:,[1,2]]
# states_abbr.columns = ["State_Country", "Population"]
# countries_states = countries.append(states.iloc[:,[0,2]]).append(states_abbr)
# countries_states = countries_states.drop([4])
# countries_states.head()

In [244]:
# sf_users.Location = [x.split(",") if x is not np.nan else np.nan for x in sf_users.Location]

# sf_users["State_Country"] = [sf_users.Location.iloc[x][-1].strip() 
#                              if sf_users.Location.iloc[x] is not np.nan else np.nan
#                           for x in np.arange(0,len(sf_users))]

In [245]:
sf_users["Country_State"] = [x.split(",") if x is not np.nan else np.nan for x in sf_users.Location] 
sf_users["Country_State"] = [sf_users.Country_State.iloc[x][-1] if sf_users.Country_State.iloc[x] is not np.nan else np.nan for x in np.arange(0,len(sf_users))]

states_dict = {abbrev: state_name for abbrev, state_name in zip(states.Abbr, states.State_Country)} 
countries_set = {country for country in countries.State_Country}

garbage = set() 
n_garbage = 0 
def abbrev_to_state(word): 
    global n_garbage 
    """If word is a country or state name it is left unchagned If state abbreviation it is changed to state name If anything else it is reported and thrown out""" 
    if word is np.nan: 
        return np.nan 
    stripped_word = word.strip() 
    if stripped_word in states_dict: 
        return states_dict[stripped_word] 
    elif stripped_word in states_dict.values(): 
        return stripped_word 
    elif stripped_word in countries_set: 
        return stripped_word 
    elif stripped_word.lower() == "usa": 
        return "United States" 
    else: 
        n_garbage += 1 
        garbage.add(stripped_word) 
        return np.nan
sf_users["Country_cleaned"] = sf_users.Country_State.map(lambda word: abbrev_to_state(word))

def change_to_usa(country_state): 
    if country_state in states_dict.values(): 
        return "United States" 
    else: 
        return country_state
sf_users.Country_cleaned = sf_users.Country_cleaned.map(lambda x: change_to_usa(x))

In [ ]:
#sf_users_with_population = pd.merge(sf_users, countries_states, on_left = "State_Country", , how = "left")

In [254]:
Users_grouped = sf_users.groupby("Country_cleaned").size()
Users_grouped = Users_grouped.reset_index()
Users_grouped.columns = ["Country", "Count"]
Users_grouped = Users_grouped.sort_values("Count", ascending=False).reset_index(drop=True)
Users_grouped.head()

,Country,Count
0,United States,8652
1,India,2226
2,United Kingdom,1730
3,Canada,961
4,Germany,958


In [255]:
countries_codes = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/2014_world_gdp_with_codes.csv')
countries_codes.head()

,COUNTRY,GDP (BILLIONS),CODE
0,Afghanistan,21.71,AFG
1,Albania,13.40,ALB
2,Algeria,227.80,DZA
3,American Samoa,0.75,ASM
4,Andorra,4.80,AND


In [258]:
Users_grouped2 = pd.merge(Users_grouped, countries_codes.loc[:,["COUNTRY", "CODE"]], 
                         left_on="Country", right_on="COUNTRY", how="left")
Users_grouped2.head()

,Country,Count,COUNTRY,CODE
0,United States,8652,United States,USA
1,India,2226,India,IND
2,United Kingdom,1730,United Kingdom,GBR
3,Canada,961,Canada,CAN
4,Germany,958,Germany,DEU


In [261]:
#!pip3 install plotly

In [270]:
import plotly
import plotly.plotly as py
import pandas as pd


plotly.tools.set_credentials_file(username='Hedwiga', api_key='2vu5NW9wEmxmao9RmKws')

df = Users_grouped2

data = [ dict(
        type = 'choropleth',
        locations = df['CODE'],
        z = df['Count'],
        text = df['COUNTRY'],
        colorscale = [[0,"rgb(5, 10, 172)"],[0.35,"rgb(40, 60, 190)"],[0.5,"rgb(70, 100, 245)"],\
            [0.6,"rgb(90, 120, 245)"],[0.7,"rgb(106, 137, 247)"],[1,"rgb(220, 220, 220)"]],
        autocolorscale = False,
        reversescale = True,
        marker = dict(
            line = dict (
                color = 'rgb(180,180,180)',
                width = 0.5
            ) ),
        colorbar = dict(
            autotick = False,
            tickprefix = '$',
            title = 'GDP<br>Billions US$'),
      ) ]

layout = dict(
    title = 'Mapa',
    geo = dict(
        showframe = False,
        showcoastlines = False,
        projection = dict(
            type = 'Mercator'
        )
    )
)

fig = dict( data=data, layout=layout )
py.iplot( fig, validate=False, filename='d3-world-map' )

#https://plot.ly/python/choropleth-maps/

High five! You successfully sent some data to your account on plotly. View your plot in your browser at https://plot.ly/~Hedwiga/0 or inside your plot.ly account where it is named 'd3-world-map'
